# Text Summarization with attention mechanism

In [18]:
import pandas as pd
import numpy as np
import re # regular expression
import warnings
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup



In [3]:
data = pd.read_csv(
  '../Dataset//amazon_reviews/Reviews.csv',
)
data.head()
data.info(
  verbose=True,
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


## Explonatory data analysis and data preparation

We need to remove all duplicate sentences from the dataset.\
Remove and NA values from the dataset.

```python

In [62]:
data.drop_duplicates(subset=['Text'], inplace=True)# drop duplicates
data.dropna(inplace=True) # drop na values

data.info(
  verbose=True,
)

data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393217 entries, 0 to 568453
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      393217 non-null  int64 
 1   ProductId               393217 non-null  object
 2   UserId                  393217 non-null  object
 3   ProfileName             393217 non-null  object
 4   HelpfulnessNumerator    393217 non-null  int64 
 5   HelpfulnessDenominator  393217 non-null  int64 
 6   Score                   393217 non-null  int64 
 7   Time                    393217 non-null  int64 
 8   Summary                 393217 non-null  object
 9   Text                    393217 non-null  object
 10  cleaned_text            393217 non-null  object
 11  cleaned_summary         393217 non-null  object
dtypes: int64(5), object(7)
memory usage: 39.0+ MB


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleaned_text,cleaned_summary
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,bought several vitality canned dog food produc...,good quality dog food
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,product arrived labeled jumbo salted peanuts p...,not as advertised
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,confection around centuries light pillowy citr...,delight says it all
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,looking secret ingredient robitussin believe f...,cough medicine
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...,great taffy


In [61]:
data.tail(50)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleaned_text,cleaned_summary
568379,568380,B000H28ABW,A1XVKAMXSOW8ZV,"C. E. Hunsicker ""Hatshepsut""",2,3,3,1256515200,Just OK,I hadn't tried this taco seasoning prior to pu...,tried taco seasoning prior purchasing bulk lim...,just ok
568380,568381,B000H28ABW,A25RYX4NFWDJ2N,"Lyvonne Hipperling ""HEALTH ADVOCATE""",2,3,5,1181692800,HEALTHY EATING,THIS TACO SEASONING WAS RECOMMENDED TO ME AS A...,taco seasoning recommended natural seasoning u...,healthy eating
568381,568382,B000H28ABW,A1I44EGA0EDJ03,L. Woods,0,0,5,1351036800,Best taco seasoning you can buy,Notice that this taco seasoning packet weighs ...,notice taco seasoning packet weighs much brand...,best taco seasoning you can buy
568382,568383,B000H28ABW,A1TZXENU9QMGQE,nita,0,0,5,1350259200,Best buy 4 your money,I purchased a packet of the taco seasoning and...,purchased packet taco seasoning suprise best o...,best buy your money
568383,568384,B000H28ABW,A2RM8BU1FCYQM9,"Former Oak Parker ""music buff""",0,0,5,1324339200,Best Taco Seasoning EVER!!!,Wick Fowler Taco Seasoning is by far the best ...,wick fowler taco seasoning far best tasting ta...,best taco seasoning ever
568384,568385,B000H28ABW,A8T6W1M15FM75,"CA-SK ""CLA""",0,3,2,1255219200,Bland Taco Seasoning,"Tasteless, bland, insipid - how many synonyms ...",tasteless bland insipid many synonyms need des...,bland taco seasoning
568385,568386,B005OCX5XI,A6RFVZFCCPPAB,Sandra Thomas,1,1,1,1345507200,Rip-off: Be careful before u buy!!!!!!!,"Be very careful to check out the the oz size, ...",careful check oz size picture deceiving impres...,rip off be careful before buy
568404,568405,B005OTVL8C,A1UJMVE0LCOI45,J. Wolf,0,0,5,1350172800,great flavor boost,If you want to add a hint of Meyer lemon flavo...,want add hint meyer lemon flavor dish add driz...,great flavor boost
568405,568406,B000P56I7Y,A2ZKNGL20FBK2I,Shatzi Crabtree,3,5,5,1274918400,Arrived Fine,The product arrived pretty quickly and was eas...,product arrived pretty quickly easy make taste...,arrived fine
568406,568407,B0039KE8Y2,A392FNHBJR1IY6,Leo Breeden,0,0,4,1336867200,great taste,This apple butter has a great taste but the pr...,apple butter great taste price would purchase ...,great taste


#### Some words will be in its short form, which will not be recognized by algorithm. 
#### We will now replace these words by their exact form

In [39]:

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}


#copied from stackoverflow

In [40]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/jeel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
stop_words = set(stopwords.words('english')) 

def text_cleaner(text, num):
    newString = text.lower()  # Convert everything to lowercase
    newString = re.sub('<[^<]+?>', '', newString) #Remove HTML tags
    newString = re.sub(r'\([^)]*\)', '', newString)     #Remove parenthesis
    newString = re.sub('"','', newString)   #Remove quotes
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")]) #Contraction mapping
    newString = re.sub(r"'s\b","",newString)   #Remove all 's
    newString = re.sub("[^a-zA-Z]", " ", newString)  # remove all character except letters
    newString = re.sub('[m]{2,}', 'mm', newString)  # remove all m's
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:    #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()


#using stackoverflow and regex documentation to clean the data

In [64]:
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0))

In [55]:
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

In [57]:
cleaned_text[:10]

['bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better',
 'product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo',
 'confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story lewis lion witch wardrobe treat seduces edmund selling brother sisters witch',
 'looking secret ingredient robitussin believe found got addition root beer extract ordered made cherry soda flavor medicinal',
 'great taffy great price wide assortment yummy taffy delivery quick taffy lover deal',
 'got wild hair taffy ordered five pound bag taffy enjoyable many flavors watermelon root beer melon peppermint grape etc complaint bit much red black licorice flavored pieces kids husband l

# ergeter

In [58]:
cleaned_summary[:10]

['good quality dog food',
 'not as advertised',
 'delight says it all',
 'cough medicine',
 'great taffy',
 'nice taffy',
 'great just as good as the expensive brands',
 'wonderful tasty taffy',
 'yay barley',
 'healthy dog food']

In [59]:

data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary

In [60]:
# drop empty rows

data.replace('', np.nan, inplace=True)
data.dropna(axis=0,inplace=True)